In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from yahoo_fin import stock_info as si
from collections import deque

import os
import numpy as np
import pandas as pd
import random

In [2]:
# set seed, so we can get the same results after rerunning several times
np.random.seed(123)
tf.random.set_seed(123)
random.seed(123)

In [3]:
def shuffle_in_unison(a, b):
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

In [4]:
def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
                #adjclose:adjusted close value
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):

    if isinstance(ticker, str):
        # load ticker symbol from yahoo_fin
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
        
    # add date as a column
    if "date" not in df.columns:
        df["date"] = df.index
        
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
        
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    
    """
    last `lookup_step` columns contains NaN in future column
    get them before droping NaNs
    """
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    
    # drop NaNs
    df.dropna(inplace=True)
    
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    """
    get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    this last_sequence will be used to predict future stock prices that are not available in the dataset
    """        
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)    
    # add to result
    result['last_sequence'] = last_sequence
    
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    if split_by_date:
        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        
        if shuffle:
            # shuffle the datasets for training (if shuffle parameter is set)
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
            
    else:    
        # split the dataset randomly
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                                test_size=test_size, shuffle=shuffle)
    # get the list of test set dates
    dates = result["X_test"][:, -1, -1]
    # retrieve test features from the original dataframe
    result["test_df"] = result["df"].loc[dates]
    # remove duplicated dates in the testing dataframe
    result["test_df"] = result["test_df"][~result["test_df"].index.duplicated(keep='first')]
    # remove dates from the training/testing sets & convert to float32
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

In [5]:
def create_model(sequence_length, n_features, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), batch_input_shape=(None, sequence_length, n_features)))
            else:
                model.add(cell(units, return_sequences=True, batch_input_shape=(None, sequence_length, n_features)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [6]:
import os
import time
from tensorflow.keras.layers import LSTM

# Window size or the sequence length
N_STEPS = 50
# Lookup step, 1 is the next day
LOOKUP_STEP = 15
# whether to scale feature columns & output price as well
SCALE = True
scale_str = f"sc-{int(SCALE)}"
# whether to shuffle the dataset
SHUFFLE = True
shuffle_str = f"sh-{int(SHUFFLE)}"
# whether to split the training/testing set by date
SPLIT_BY_DATE = False
split_by_date_str = f"sbd-{int(SPLIT_BY_DATE)}"
# test ratio size, 0.2 is 20%
TEST_SIZE = 0.2
# features to use
FEATURE_COLUMNS = ["adjclose", "volume", "open", "high", "low"]
# date now
date_now = time.strftime("%Y-%m-%d")
### model parameters
N_LAYERS = 2
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 256
# 40% dropout
DROPOUT = 0.4
# whether to use bidirectional RNNs
BIDIRECTIONAL = False
### training parameters
# mean absolute error loss
# LOSS = "mae"
# huber loss
LOSS = "huber_loss"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 500
# tesla stock market
ticker = "TSLA"
ticker_data_filename = os.path.join("data", f"{ticker}_{date_now}.csv")
# model name to save, making it as unique as possible based on parameters
model_name = f"{date_now}_{ticker}-{shuffle_str}-{scale_str}-{split_by_date_str}-\
{LOSS}-{OPTIMIZER}-{CELL.__name__}-seq-{N_STEPS}-step-{LOOKUP_STEP}-layers-{N_LAYERS}-units-{UNITS}"
if BIDIRECTIONAL:
    model_name += "-b"

In [7]:
# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")

In [8]:
# load the data
data = load_data(ticker, N_STEPS, scale=SCALE, split_by_date=SPLIT_BY_DATE, 
                shuffle=SHUFFLE, lookup_step=LOOKUP_STEP, test_size=TEST_SIZE, 
                feature_columns=FEATURE_COLUMNS)
# save the dataframe
data["df"].to_csv(ticker_data_filename)
# construct the model
model = create_model(N_STEPS, len(FEATURE_COLUMNS), loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=OPTIMIZER, bidirectional=BIDIRECTIONAL)
# some tensorflow callbacks
checkpointer = ModelCheckpoint(os.path.join("results", model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
# train the model and save the weights whenever we see 
# a new optimal model using ModelCheckpoint
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[checkpointer, tensorboard],
                    verbose=1)

Epoch 1/500
34/34 [==============================] - 5s 44ms/step - loss: 0.0043 - mean_absolute_error: 0.0440 - val_loss: 6.3634e-04 - val_mean_absolute_error: 0.0171

Epoch 00001: val_loss improved from inf to 0.00064, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 2/500
34/34 [==============================] - 1s 15ms/step - loss: 6.0460e-04 - mean_absolute_error: 0.0177 - val_loss: 7.0959e-04 - val_mean_absolute_error: 0.0209

Epoch 00002: val_loss did not improve from 0.00064
Epoch 3/500
34/34 [==============================] - 1s 15ms/step - loss: 8.9925e-04 - mean_absolute_error: 0.0200 - val_loss: 5.6720e-04 - val_mean_absolute_error: 0.0170

Epoch 00003: val_loss improved from 0.00064 to 0.00057, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 4/500
34/34 [==============================] - 1s 15ms/step - loss: 8.2375e-04 - mean_absolu

34/34 [==============================] - 1s 15ms/step - loss: 7.2527e-04 - mean_absolute_error: 0.0184 - val_loss: 6.1645e-04 - val_mean_absolute_error: 0.0201

Epoch 00032: val_loss did not improve from 0.00045
Epoch 33/500
34/34 [==============================] - 1s 15ms/step - loss: 7.6469e-04 - mean_absolute_error: 0.0195 - val_loss: 0.0010 - val_mean_absolute_error: 0.0254

Epoch 00033: val_loss did not improve from 0.00045
Epoch 34/500
34/34 [==============================] - 1s 16ms/step - loss: 0.0012 - mean_absolute_error: 0.0273 - val_loss: 7.2089e-04 - val_mean_absolute_error: 0.0155

Epoch 00034: val_loss did not improve from 0.00045
Epoch 35/500
34/34 [==============================] - 1s 15ms/step - loss: 8.7525e-04 - mean_absolute_error: 0.0190 - val_loss: 4.2750e-04 - val_mean_absolute_error: 0.0138

Epoch 00035: val_loss improved from 0.00045 to 0.00043, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Ep

34/34 [==============================] - 1s 15ms/step - loss: 5.8345e-04 - mean_absolute_error: 0.0156 - val_loss: 4.2604e-04 - val_mean_absolute_error: 0.0127

Epoch 00062: val_loss did not improve from 0.00027
Epoch 63/500
34/34 [==============================] - 1s 15ms/step - loss: 4.9805e-04 - mean_absolute_error: 0.0144 - val_loss: 4.1008e-04 - val_mean_absolute_error: 0.0127

Epoch 00063: val_loss did not improve from 0.00027
Epoch 64/500
34/34 [==============================] - 1s 15ms/step - loss: 4.5845e-04 - mean_absolute_error: 0.0151 - val_loss: 7.6949e-04 - val_mean_absolute_error: 0.0159

Epoch 00064: val_loss did not improve from 0.00027
Epoch 65/500
34/34 [==============================] - 1s 15ms/step - loss: 7.1537e-04 - mean_absolute_error: 0.0182 - val_loss: 4.4048e-04 - val_mean_absolute_error: 0.0147

Epoch 00065: val_loss did not improve from 0.00027
Epoch 66/500
34/34 [==============================] - 1s 15ms/step - loss: 6.2090e-04 - mean_absolute_error: 0.01

34/34 [==============================] - 1s 15ms/step - loss: 7.0573e-04 - mean_absolute_error: 0.0167 - val_loss: 3.0756e-04 - val_mean_absolute_error: 0.0113

Epoch 00097: val_loss did not improve from 0.00021
Epoch 98/500
34/34 [==============================] - 1s 15ms/step - loss: 5.3488e-04 - mean_absolute_error: 0.0156 - val_loss: 2.3626e-04 - val_mean_absolute_error: 0.0111

Epoch 00098: val_loss did not improve from 0.00021
Epoch 99/500
34/34 [==============================] - 1s 15ms/step - loss: 3.7806e-04 - mean_absolute_error: 0.0140 - val_loss: 1.8069e-04 - val_mean_absolute_error: 0.0097

Epoch 00099: val_loss improved from 0.00021 to 0.00018, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 100/500
34/34 [==============================] - 1s 15ms/step - loss: 3.8265e-04 - mean_absolute_error: 0.0137 - val_loss: 3.9121e-04 - val_mean_absolute_error: 0.0114

Epoch 00100: val_loss did not improve from 0

Epoch 132/500
34/34 [==============================] - 1s 15ms/step - loss: 2.8394e-04 - mean_absolute_error: 0.0122 - val_loss: 2.7850e-04 - val_mean_absolute_error: 0.0101

Epoch 00132: val_loss did not improve from 0.00017
Epoch 133/500
34/34 [==============================] - 1s 16ms/step - loss: 3.9132e-04 - mean_absolute_error: 0.0131 - val_loss: 3.7320e-04 - val_mean_absolute_error: 0.0127

Epoch 00133: val_loss did not improve from 0.00017
Epoch 134/500
34/34 [==============================] - 1s 15ms/step - loss: 4.8788e-04 - mean_absolute_error: 0.0145 - val_loss: 1.8703e-04 - val_mean_absolute_error: 0.0090

Epoch 00134: val_loss did not improve from 0.00017
Epoch 135/500
34/34 [==============================] - 1s 15ms/step - loss: 2.7029e-04 - mean_absolute_error: 0.0127 - val_loss: 3.9059e-04 - val_mean_absolute_error: 0.0118

Epoch 00135: val_loss did not improve from 0.00017
Epoch 136/500
34/34 [==============================] - 1s 15ms/step - loss: 3.6413e-04 - mean_ab

...

34/34 [==============================] - 1s 15ms/step - loss: 1.3048e-04 - mean_absolute_error: 0.0095 - val_loss: 9.6924e-05 - val_mean_absolute_error: 0.0075

Epoch 00343: val_loss did not improve from 0.00009
Epoch 344/500
34/34 [==============================] - 1s 16ms/step - loss: 1.5472e-04 - mean_absolute_error: 0.0097 - val_loss: 9.7222e-05 - val_mean_absolute_error: 0.0077

Epoch 00344: val_loss did not improve from 0.00009
Epoch 345/500
34/34 [==============================] - 1s 15ms/step - loss: 1.5044e-04 - mean_absolute_error: 0.0099 - val_loss: 9.2505e-05 - val_mean_absolute_error: 0.0078

Epoch 00345: val_loss did not improve from 0.00009
Epoch 346/500
34/34 [==============================] - 1s 16ms/step - loss: 1.4735e-04 - mean_absolute_error: 0.0096 - val_loss: 1.0999e-04 - val_mean_absolute_error: 0.0081

Epoch 00346: val_loss did not improve from 0.00009
Epoch 347/500
34/34 [==============================] - 1s 15ms/step - loss: 1.4678e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 2.6435e-04 - mean_absolute_error: 0.0118 - val_loss: 1.5439e-04 - val_mean_absolute_error: 0.0092

Epoch 00379: val_loss did not improve from 0.00009
Epoch 380/500
34/34 [==============================] - 1s 15ms/step - loss: 2.6226e-04 - mean_absolute_error: 0.0121 - val_loss: 1.1320e-04 - val_mean_absolute_error: 0.0078

Epoch 00380: val_loss did not improve from 0.00009
Epoch 381/500
34/34 [==============================] - 1s 16ms/step - loss: 2.8287e-04 - mean_absolute_error: 0.0123 - val_loss: 1.0452e-04 - val_mean_absolute_error: 0.0080

Epoch 00381: val_loss did not improve from 0.00009
Epoch 382/500
34/34 [==============================] - 1s 15ms/step - loss: 2.1390e-04 - mean_absolute_error: 0.0113 - val_loss: 9.5718e-05 - val_mean_absolute_error: 0.0078

Epoch 00382: val_loss did not improve from 0.00009
Epoch 383/500
34/34 [==============================] - 1s 15ms/step - loss: 1.5024e-04 - mean_absolute_error: 

34/34 [==============================] - 1s 16ms/step - loss: 1.5864e-04 - mean_absolute_error: 0.0102 - val_loss: 8.1717e-05 - val_mean_absolute_error: 0.0070

Epoch 00414: val_loss improved from 0.00009 to 0.00008, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 415/500
34/34 [==============================] - 1s 15ms/step - loss: 1.4346e-04 - mean_absolute_error: 0.0099 - val_loss: 1.1144e-04 - val_mean_absolute_error: 0.0075

Epoch 00415: val_loss did not improve from 0.00008
Epoch 416/500
34/34 [==============================] - 1s 15ms/step - loss: 1.3095e-04 - mean_absolute_error: 0.0092 - val_loss: 1.2928e-04 - val_mean_absolute_error: 0.0075

Epoch 00416: val_loss did not improve from 0.00008
Epoch 417/500
34/34 [==============================] - 1s 15ms/step - loss: 1.3413e-04 - mean_absolute_error: 0.0094 - val_loss: 1.0455e-04 - val_mean_absolute_error: 0.0089

Epoch 00417: val_loss did not improve from

34/34 [==============================] - 1s 15ms/step - loss: 1.3581e-04 - mean_absolute_error: 0.0101 - val_loss: 9.0234e-05 - val_mean_absolute_error: 0.0072

Epoch 00449: val_loss did not improve from 0.00008
Epoch 450/500
34/34 [==============================] - 1s 15ms/step - loss: 1.6414e-04 - mean_absolute_error: 0.0100 - val_loss: 9.3509e-05 - val_mean_absolute_error: 0.0080

Epoch 00450: val_loss did not improve from 0.00008
Epoch 451/500
34/34 [==============================] - 1s 15ms/step - loss: 1.2909e-04 - mean_absolute_error: 0.0093 - val_loss: 8.6850e-05 - val_mean_absolute_error: 0.0067

Epoch 00451: val_loss did not improve from 0.00008
Epoch 452/500
34/34 [==============================] - 1s 15ms/step - loss: 1.5471e-04 - mean_absolute_error: 0.0096 - val_loss: 1.0387e-04 - val_mean_absolute_error: 0.0078

Epoch 00452: val_loss did not improve from 0.00008
Epoch 453/500
34/34 [==============================] - 1s 15ms/step - loss: 1.7387e-04 - mean_absolute_error: 


Epoch 00483: val_loss did not improve from 0.00007
Epoch 484/500
34/34 [==============================] - 1s 15ms/step - loss: 1.6789e-04 - mean_absolute_error: 0.0095 - val_loss: 9.0449e-05 - val_mean_absolute_error: 0.0070

Epoch 00484: val_loss did not improve from 0.00007
Epoch 485/500
34/34 [==============================] - 1s 16ms/step - loss: 1.2105e-04 - mean_absolute_error: 0.0086 - val_loss: 1.1836e-04 - val_mean_absolute_error: 0.0078

Epoch 00485: val_loss did not improve from 0.00007
Epoch 486/500
34/34 [==============================] - 1s 15ms/step - loss: 1.3798e-04 - mean_absolute_error: 0.0090 - val_loss: 6.7253e-05 - val_mean_absolute_error: 0.0066

Epoch 00486: val_loss improved from 0.00007 to 0.00007, saving model to results\2021-03-31_TSLA-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 487/500
34/34 [==============================] - 1s 15ms/step - loss: 1.6222e-04 - mean_absolute_error: 0.0092 - val_loss: 9.8462e-05 - val_mean_